In [ ]:
from IPython.display import HTML
import time

handle = display(HTML("""<marquee>👌</marquee>"""), display_id='html_marquee1')
time.sleep(2)
handle = display(HTML("""<marquee>🧠 Can a computer learn complex, abstract tasks from just a few examples?</marquee>"""), display_id='html_marquee1', update=True)

<p style="border-bottom: 5px solid red"></p>
<p style="border-bottom: 15px solid darkcyan"></p>

# <div style="color:yellow;display:inline-block;border-radius:5px;background-color:lightgray;font-block:Nexa;overflow:hidden"><p style="padding:15px;color:darkred;overflow:hidden;font-size:70%;letter-spacing:0.5px;margin:0"><b> </b>✍️Description of this Notebook :</p></div>

- The ARC challenge is very different from other Kaggle challenges and participants have limited resources to study and prepare themselves. That's why we decided to prepare and publish the **"Tutorial-1"** notebook. Of course, if this notebook is of interest and used by readers, maybe we can publish more "Tutorial" notebooks.

- The ARC challenge is the initial steps for machine learning in the field of **"Abstraction and Reasoning"**. Therefore, we should not expect that unique routes are recommended for participating in this challenge. In this connection, we published discussion titled ["ARC2020 Vs. ARC2024"](https://www.kaggle.com/competitions/arc-prize-2024/discussion/521200) in "Discussion". We recommend that you read that discussion.

- In this notebook, you will first get to know the tasks of this challenge and you will be provided with two functions at the same time. function **hello_arc( )** which displays your desired task in detail and function **get_arc( )** which returns different values of each task and you can test your solvers with these values.

- Then we will check the types of **Mirror&Rotate** in the images with some examples and you can get to know some practical solvers in this field.

- In the next parts, with numerous examples, you will get to know solvers that solve other topics and tasks. For example: **"Select Zone" and "Sudoku Puzzles" and "Carpet Repair" and "Identical Columns & Rows"**.

- The next function is called **set_tasks_solvers( )**, by writing a number of solvers and using this function, you can test how many tasks of the corpus can be solved correctly by these solvers. You can read the description of this function in its own section.

- The last function is called **set_submission( )**, by using this function you can select the correct answers of the solvers to the "testing" tasks and prepare them for submission. You can read the description of this function in its own section.

- Also, at the end of the notebook, we provided a solution for the proposed **practice** to show that you can easily add this solver, as well as any other solver that you write yourself, to the list of solvers in this notebook.

- **Please pay attention**: The number of solvers is still very small, but about five percent of the tasks in the corpus can be solved with these solvers. This is the beginning and for now we are using the "Brute Force" method. But this method should give way to a smart method. It is only then that we have taken a serious step towards learning "Abstraction and Reasoning" in machines. In other words, we first create a number of keys (solvers) and then to open (solve) each task, the machine must be able to choose the right key (or a number of keys that are more suitable) among all the keys. But in the "Brute Force" method, all keys are tested to find the right key, if any.


<p style="border-bottom: 5px solid red"></p>
<p style="border-bottom: 15px solid darkcyan"></p>

In [ ]:
import os, gc
import sys, pdb
import copy, time
import json, random

import itertools
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats
from pathlib import Path

import matplotlib
from matplotlib import colors
import matplotlib.pyplot as plt
from colorama import Style, Fore
%matplotlib inline

In [ ]:
!ls ../input/*

In [ ]:
train1_path = '../input/arc-prize-2024/arc-agi_training_challenges.json'
train2_path = '../input/arc-prize-2024/arc-agi_training_solutions.json'

eval1_path = '../input/arc-prize-2024/arc-agi_evaluation_challenges.json'
eval2_path = '../input/arc-prize-2024/arc-agi_evaluation_solutions.json'

test_path = '../input/arc-prize-2024/arc-agi_test_challenges.json'
sample_path = '../input/arc-prize-2024/sample_submission.json'

# ......................................................................................................
cmap = colors.ListedColormap(
    ['#000000', '#0074D9','#FF4136','#2ECC40','#FFDC00',
     '#AAAAAA', '#F012BE', '#FF851B', '#7FDBFF', '#870C25'])

norm = colors.Normalize(vmin=0, vmax=9)
color_list = ["black", "blue", "red", "green", "yellow", "gray", "magenta", "orange", "sky", "brown"]

# ......................................................................................................
plt.figure(figsize=(5, 2), dpi=200)
plt.imshow([list(range(10))], cmap=cmap, norm=norm)
plt.xticks(list(range(10)))
plt.yticks([])
plt.show()

# ......................................................................................................


![](https://cdn-images-1.medium.com/max/1000/1*D2PZiUpNx4PSSv-QHOJx0Q.png)

# <span style="color:darkcyan;">🧩Understanding Tasks</span>

<p style="border-bottom: 5px solid red"></p>
<p style="border-bottom: 15px solid darkcyan"></p>

### Grid:

- A "grid" is a rectangular matrix (list of lists) of integers between 0 and 9 (inclusive). The smallest possible grid size is 1x1 and the largest is 30x30.

- For better visualization, we conventionally assign ten different colors to ten numbers (zero to nine).

- The selected colors are "black", "blue", "red", "green", "yellow", "gray", "magenta", "orange", "sky", "brown".

### Pair:

- A "pair" is a dictionary with two fields:

> "**input**": the input "grid" for the pair.
> 
> "**output**": the output "grid" for the pair.

### Task:

- Each task contains a dictionary with two fields:

> "**train**": demonstration input/output pairs. It is a list of "pairs".
> 
> "**test**": test input - your model should predict the output.

### Training (challenges & solutions):

- It contains four hundred tasks.

### Evaluation (challenges & solutions):

- It contains four hundred tasks.

### Testing (challenges):

- Contains a hundred tasks.


<p style="border-bottom: 5px solid red"></p>

# 🔎hellow_arc( )

- We wrote a simple function called hello_arc( ) which takes the task number and the file name (trainnig or evaluation or testing) and returns the description and image of the task.

- If (values=True), this function also returns the numeric values ​​of the task, otherwise, it returns only the images of the task.

- In the following, you will get to know more about this function and the tasks of this challenge with some examples.


In [ ]:
# ::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::
def plot_pic(x):
    plt.imshow(np.array(x), cmap=cmap, norm=norm)
    plt.show()
    
# ::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::
def plot_data(task):
    n = len(task["train"]) + len(task["test"])
    fig, axs = plt.subplots(2, n, figsize=(4*n,8), dpi=200)
    plt.subplots_adjust(wspace=0, hspace=0)
    fig_num = 0
    for i, t in enumerate(task["train"]):
        t_in, t_out = np.array(t["input"]), np.array(t["output"])
        axs[0][fig_num].imshow(t_in, cmap=cmap, norm=norm)
        axs[0][fig_num].set_title(f'Train-{i} in')
        axs[0][fig_num].set_yticks(list(range(t_in.shape[0])))
        axs[0][fig_num].set_xticks(list(range(t_in.shape[1])))
        axs[1][fig_num].imshow(t_out, cmap=cmap, norm=norm)
        axs[1][fig_num].set_title(f'Train-{i} out')
        axs[1][fig_num].set_yticks(list(range(t_out.shape[0])))
        axs[1][fig_num].set_xticks(list(range(t_out.shape[1])))
        fig_num += 1
    for i, t in enumerate(task["test"]):
        t_in, t_out = np.array(t["input"]), np.array(t["output"])
        axs[0][fig_num].imshow(t_in, cmap=cmap, norm=norm)
        axs[0][fig_num].set_title(f'Test-{i} in')
        axs[0][fig_num].set_yticks(list(range(t_in.shape[0])))
        axs[0][fig_num].set_xticks(list(range(t_in.shape[1])))
        axs[1][fig_num].imshow(t_out, cmap=cmap, norm=norm)
        axs[1][fig_num].set_title(f'Test-{i} out')
        axs[1][fig_num].set_yticks(list(range(t_out.shape[0])))
        axs[1][fig_num].set_xticks(list(range(t_out.shape[1])))
        fig_num += 1
    
    plt.tight_layout()
    plt.show()
    
# ::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::
def plot_task(task):
    n = len(task["train"]) + len(task["test"])
    fig, axs = plt.subplots(2, n, figsize=(4*n,8), dpi=200)
    plt.subplots_adjust(wspace=0, hspace=0)
    fig_num = 0
    for i, t in enumerate(task["train"]):
        t_in, t_out = np.array(t["input"]), np.array(t["output"])
        axs[0][fig_num].imshow(t_in, cmap=cmap, norm=norm)
        axs[0][fig_num].set_title(f'Train-{i} in')
        axs[0][fig_num].set_yticks(list(range(t_in.shape[0])))
        axs[0][fig_num].set_xticks(list(range(t_in.shape[1])))
        axs[1][fig_num].imshow(t_out, cmap=cmap, norm=norm)
        axs[1][fig_num].set_title(f'Train-{i} out')
        axs[1][fig_num].set_yticks(list(range(t_out.shape[0])))
        axs[1][fig_num].set_xticks(list(range(t_out.shape[1])))
        fig_num += 1
    for i, t in enumerate(task["test"]):
        t_in = np.array(t["input"])
        axs[0][fig_num].imshow(t_in, cmap=cmap, norm=norm)
        axs[0][fig_num].set_title(f'Test-{i} in')
        axs[0][fig_num].set_yticks(list(range(t_in.shape[0])))
        axs[0][fig_num].set_xticks(list(range(t_in.shape[1])))
        fig_num += 1
    
    plt.tight_layout()
    plt.show()  
    
# ::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::


In [ ]:

# ::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::: hellow_arc
def hellow_arc(n, name_file, values=False):
    '''
    name_file: ('tr' or 'train' or 'training') then (n=0 up to n=399)
    name_file: ('ev' or 'eval' or 'evaluation') then (n=0 up to n=399)
    name_file: ('te' or 'test' or 'testing') then (n=0 up to n=99)
    
    values=False: (Don't print task values)
    values=True: (Print task values)
    '''
    # ...............................................................................
    if (name_file =='tr') or (name_file =='train') or (name_file =='training'):
        
        with open(train1_path,'r') as f:
            train1_name = list(json.load(f).keys())

        with open(train2_path,'r') as f:
            train2_name = list(json.load(f).keys())
        
        with open(train1_path,'r') as f:
            train1_task = list(json.load(f).values())

        with open(train2_path,'r') as f:
            train2_task = list(json.load(f).values())
        
        display(pd.DataFrame(data={'Task Name': train1_name[n], 'Files': 'training_challenges'}, index=[n]))
        
        if (values):
            display(train1_task[n])
        plot_task(train1_task[n])
    
        for i in range(len(train2_task[n])):
            display(pd.DataFrame(data={'Answers for task': train2_name[n], 'Items': i, 'Files': 'training_solutions'}, index=[n]))
            
            if (values):
                display(train2_task[n][i])
            plot_pic(train2_task[n][i])         
    
    # ...............................................................................
    if (name_file =='ev') or (name_file =='eval') or (name_file =='evaluation'):
        
        with open(eval1_path,'r') as f:
            eval1_name = list(json.load(f).keys())

        with open(eval2_path,'r') as f:
            eval2_name = list(json.load(f).keys())
        
        with open(eval1_path,'r') as f:
            eval1_task = list(json.load(f).values())

        with open(eval2_path,'r') as f:
            eval2_task = list(json.load(f).values())
        
        display(pd.DataFrame(data={'Task Name': eval1_name[n], 'Files': 'evaluation_challenges'}, index=[n]))
        
        if (values):
            display(eval1_task[n])
        plot_task(eval1_task[n])
    
        for i in range(len(eval2_task[n])):
            display(pd.DataFrame(data={'Answers for task': eval2_name[n], 'Items': i, 'Files': 'evaluation_solutions'}, index=[n]))
            
            if (values):
                display(eval2_task[n][i])
            plot_pic(eval2_task[n][i])    

    # ...............................................................................
    if (name_file =='te') or (name_file =='test') or (name_file =='testing'):
        
        with open(test_path,'r') as f:
            test_name = list(json.load(f).keys())   
    
        with open(test_path,'r') as f:
            test_task = list(json.load(f).values())
           
        display(pd.DataFrame(data={'Task Name': test_name[n], 'Files': 'test_challenges'}, index=[n]))
        
        if (values):
            display(test_task[n])
        plot_task(test_task[n])
        
# ::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::: 


<p style="border-bottom: 5px solid red"></p>

# 🔎get_arc( )

- The function get_arc( ) also takes the task number and file name (training or evaluation or test) and returns "inp" and "task" and "answer".

- We use this function when we want to evaluate a solver.

In [ ]:

# ::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::: get_arc
def get_arc(n, name_file):
    '''
    name_file: ('tr' or 'train' or 'training') then (n=0 up to n=399)
    name_file: ('ev' or 'eval' or 'evaluation') then (n=0 up to n=399)
    name_file: ('te' or 'test' or 'testing') then (n=0 up to n=99)
    '''
    # ...............................................................................
    if (name_file =='tr') or (name_file =='train') or (name_file =='training'):
        
        with open(train1_path,'r') as f:
            train1_task = list(json.load(f).values())

        with open(train2_path,'r') as f:
            train2_task = list(json.load(f).values())
            
        task = train1_task[n] 
        answer = train2_task[n] 
        
        inp = []
        for i in range(len(task['test'])):
            inp_test = np.array(task['test'][i]['input'])
            inp.append(inp_test)
    
    # ...............................................................................
    if (name_file =='ev') or (name_file =='eval') or (name_file =='evaluation'):
        
        with open(eval1_path,'r') as f:
            eval1_task = list(json.load(f).values())

        with open(eval2_path,'r') as f:
            eval2_task = list(json.load(f).values())
        
        task = eval1_task[n] 
        answer = eval2_task[n] 
        
        inp = []
        for i in range(len(task['test'])):
            inp_test = np.array(task['test'][i]['input'])
            inp.append(inp_test)

    # ...............................................................................
    if (name_file =='te') or (name_file =='test') or (name_file =='testing'):
        
        with open(test_path,'r') as f:
            test_task = list(json.load(f).values())
            
        task = test_task[n] 
        answer = [] 
        
        inp = []
        for i in range(len(task['test'])):
            inp_test = np.array(task['test'][i]['input'])
            inp.append(inp_test)
        
    # ...............................................................................
   
    return inp[0], task, answer[0]
        
# ::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::: 


<p style="border-bottom: 5px solid red"></p>

### ⭐ Example 1

- Task number 185 is from the Training (challenges & solutions) file. You can choose numbers from 0 to 399 for this file.

- This task includes 10 pairs of train (input/output) and 2 pairs of test (input/output).

- We entered True in the input of the function so that the numerical value of this task is also printed.

In [ ]:
hellow_arc(185,'train', True)

<p style="border-bottom: 5px solid red"></p>

### ⭐ Example 2

- Task number 315 is from the Evaluation (challenges & solutions) file. You can choose numbers from 0 to 399 for this file.

- This task includes 3 pairs of train (input/output) and 1 pairs of test (input/output).

In [ ]:
hellow_arc(315,'eval')

<p style="border-bottom: 5px solid red"></p>

### ⭐Example 3

- Task number 82 is from the Testing (challenges & solutions) file. You can choose numbers from 0 to 99 for this file.

- This task includes 3 pairs of train (input/output) and 1 pairs of test (input).

- As you can see, there is no test_output in this file and in this challenge we have to generate it.

- Please note that the final score of this challenge is evaluated using a hundred other tasks that the participants cannot see.

In [ ]:
hellow_arc(82,'test')

# <span style="color:darkcyan;">🔄Mirror&Rotate Image</span>

<p style="border-bottom: 5px solid red"></p>
<p style="border-bottom: 15px solid darkcyan"></p>

- By rotating and mirroring each image, a total of **8 new images** (including the original image) can be produced.

- Of course, if there are any symmetry in the original image, some of these eight images will be similar. To say that each image may produce **1 to 8 images**.

- To better recall and visualize readers, we first write a simple function (**f0000**) to produce 8 different images from the original image, and with multiple examples we will show how to use the basic geometry topics in this challenge.

- In the f0000 function, the "**inp**" input is the same as the original image.

- You will see the use of "**version**" and "**solvers**" lists as well as the "**task**" input in future examples.

In [ ]:
version = []
solvers = []

# ::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::: 0 Rotation & Flip
def f0000(inp, task):
    '''
    This function changes the shape of a task via Rotation & Flip.
    '''  
    clist = []
    clist.append(inp)                                    # v0 Same
    clist.append(np.rot90(inp, 1))                       # v1 rot90
    clist.append(np.rot90(inp, 2))                       # v2 rot180
    clist.append(np.rot90(inp, 3))                       # v3 rot270
    clist.append(np.flip(inp, axis=0))                   # v4 flip_0
    clist.append(np.flip(inp, axis=1))                   # v5 flip_1
    clist.append(np.flip(np.rot90(inp, 1), axis=0))      # v6 flip_0(rot90)
    clist.append(np.flip(np.rot90(inp, 3), axis=0))      # v7 flip_0(rot270)
    
    return clist

# :::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::
version.append(8)
solvers.append(f0000)

<p style="border-bottom: 5px solid red"></p>

### ⭐Example 4

- First, we write a grid called "inp" for the initial image and then with the help of the above function we produce 8 different images.

In [ ]:
inp = [[0, 1], [2, 3], [4, 5], [6, 7], [8, 9]]

plot_pic(inp) 

In [ ]:
clist = f0000(inp, [])

for x in clist:
    plot_pic(x)

<p style="border-bottom: 5px solid red"></p>

### ⭐Example 5

- This example is the Task No. 238 of the "training" file.

- In this example, we have to show the repetition of the unique colors with a column chart.

- First, we write an elementary solver for this example and then complete that solver.

In [ ]:
hellow_arc(238,'train')

In [ ]:
# ::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::: 
def ft1(inp, task):
    '''
    This function sorts and draws all repetitions of each color.
    '''  
    uinp = np.unique(inp, return_counts= True)[0]
    n_uinp = np.unique(inp, return_counts= True)[1]
        
    uinp_sort = uinp[np.argsort(n_uinp)]
    n_uinp_sort = np.sort(n_uinp)
    
    # .......................................................
    created = np.zeros((len(uinp_sort), n_uinp_sort[-1]))
    
    for c in range(len(uinp_sort)): 
        for n in range(n_uinp_sort[c]):
            created[c, n] = uinp_sort[c]
            
    created = np.rot90(created, 3)
    return created

# :::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::

In [ ]:
inp, task, answer = get_arc(238,'train')
created = ft1(inp, task)

print(np.array_equal(created, answer))
plot_pic(created) 

### 💡Inference

- As you can see, the "ft1" solver was able to produce the correct answer, but that is not enough. Because next time, another chart may be desired and we have to generate **all possible Mirror&Rotate** for this chart.

- By the way, in this task, the background of the answer is black (number zero). But the background color may not be black in the next tasks. So, it is better to model the background color from train pairs.

- We make these two changes to the solver above and write a better solver.

- Does this new solver answer all our needs? Definitely not, but it's a good start.


In [ ]:
# ::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::: 1 SortUniqueColor
def f0001(inp, task):
    '''
    This function sorts and draws all repetitions of each color.
    '''  
    clist = []

    uinp = np.unique(inp, return_counts= True)[0]
    n_uinp = np.unique(inp, return_counts= True)[1]
        
    uinp_sort = uinp[np.argsort(n_uinp)]
    n_uinp_sort = np.sort(n_uinp)
    
    # ............................................................. 
    inpz = np.array(task['train'][0]['input'])
    outz = np.array(task['train'][0]['output'])
            
    uinpz = np.unique(inpz, return_counts= True)[0]
    uoutz = np.unique(outz, return_counts= True)[0]
            
    not_inpz = [f for f in uoutz if f not in uinpz]
    not_outz = [f for f in uinpz if f not in uoutz]

    # ............................................................. 
    try:
        created = np.full((len(uinp_sort), n_uinp_sort[-1]), not_inpz[0])
    
        for c in range(len(uinp_sort)): 
            for n in range(n_uinp_sort[c]):
                created[c, n] = uinp_sort[c]
            
        clist = f0000(created, [])   
        
    except:
        for n in range(8):
            clist.append([])      
 
    # ............................................................. 
    return clist

# :::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::
version.append(8)
solvers.append(f0001)

In [ ]:
inp, task, answer = get_arc(238,'train')
clist = f0001(inp, task)

for x in clist:
    if (len(x)!= 0):  # For when is not equal to []
        print(np.array_equal(x, answer))
        plot_pic(x) 

<p style="border-bottom: 5px solid red"></p>

### ⭐Example 6

- This example is the Task No. 375 of the "evaluation" file.

- In this example, input returns an output image that contains four copies of the input image (and its rotations).

- First, we write an elementary solver for this example and then complete that solver.

In [ ]:
hellow_arc(375,'eval')

In [ ]:
# :::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::
def ft2(inp, task):
    '''
    This function rotates the images and then concatenates them together.
    '''  
    pr1 = inp
    pr2 = np.rot90(inp, 1) 
    pr3 = np.concatenate((pr1, pr2), axis=1)
    
    # ................................................
    prn1 = np.rot90(inp, 2) 
    prn2 = np.rot90(inp, 3)
    prn3 = np.concatenate((prn1, prn2), axis=1)
    # ................................................
    
    created = np.concatenate((pr3, prn3), axis=0)
    return created

# :::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::

In [ ]:
inp, task, answer = get_arc(375,'eval')
created = ft2(inp, task)

print(np.array_equal(created, answer))
plot_pic(created) 

### 💡Inference

- As you can see, the "ft2" solver was able to produce the correct answer, but that is not enough. Because the type of rotation in other tasks can be completely different, or even the output image in another task only contains two different versions of the input.

- With the changes in the above solver, we wrote a better solver that creates a total of 896 different versions, which certainly includes the example above.

- You can write more complete versions. Just remember to use "try" and "except" so that the solver doesn't error for different shapes. In addition, by doing this, you make sure that the length of the "clist" list is fixed at 896. This will help you later.

In [ ]:
# ::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::: 2   Catenat_2ele_2ele
def f0002(inp, task):
    '''
    This function concatenates two versions (2+2) of the same image with different rotations.
    '''   
    clist = []
    alist = f0000(inp, [])
    
    # .............................................................  
    for x in alist:
        for y in alist:
            
            try:
                pred = np.concatenate((x, y), axis=0)
                clist.append(pred) 
            except:
                clist.append([])    
 

            try:
                pred = np.concatenate((x, y), axis=1)
                clist.append(pred) 
            except:
                clist.append([])     
                
       
            try:
                pred = np.concatenate((x, y), axis=0)
                pred = np.concatenate((pred, pred), axis=1)
                clist.append(pred) 
            except:
                clist.append([])      
  

            try:
                pred = np.concatenate((x, y), axis=1)
                pred = np.concatenate((pred, pred), axis=0)
                clist.append(pred) 
            except:
                clist.append([])   
                
 
            try:
                pred1 = np.concatenate((x, y), axis=0)
                pred2 = np.concatenate((y, x), axis=0)
                pred  = np.concatenate((pred1, pred2), axis=1)
                clist.append(pred) 
            except:
                clist.append([])      
  

            try:
                pred1 = np.concatenate((x, y), axis=1)
                pred2 = np.concatenate((y, x), axis=1)
                pred  = np.concatenate((pred1, pred2), axis=0)
                clist.append(pred) 
            except:
                clist.append([])   # v0 ~ v383

            # .........................................................
            
            try:
                pred1 = np.concatenate((x, y), axis=0)
                pred2 = np.flip(pred1, axis=0)
                pred  = np.concatenate((pred1, pred2), axis=1)
                clist.append(pred) 
            except:
                clist.append([])      
                
            
            try:
                pred1 = np.concatenate((x, y), axis=0)
                pred2 = np.flip(pred1, axis=1)
                pred  = np.concatenate((pred1, pred2), axis=1)
                clist.append(pred) 
            except:
                clist.append([])    

                
            try:
                pred1 = np.concatenate((x, y), axis=1)
                pred2 = np.flip(pred1, axis=0)
                pred  = np.concatenate((pred1, pred2), axis=0)
                clist.append(pred) 
            except:
                clist.append([])   
                
            
            try:
                pred1 = np.concatenate((x, y), axis=1)
                pred2 = np.flip(pred1, axis=1)
                pred  = np.concatenate((pred1, pred2), axis=0)
                clist.append(pred) 
            except:
                clist.append([])   
            
            
            try:
                pred1 = np.concatenate((x, y), axis=0)
                pred2 = np.flip(pred1, axis=0)
                pred  = np.concatenate((pred2, pred1), axis=1)
                clist.append(pred) 
            except:
                clist.append([])      
                
                
            try:
                pred1 = np.concatenate((x, y), axis=0)
                pred2 = np.flip(pred1, axis=1)
                pred  = np.concatenate((pred2, pred1), axis=1)
                clist.append(pred) 
            except:
                clist.append([]) 

                
            try:
                pred1 = np.concatenate((x, y), axis=1)
                pred2 = np.flip(pred1, axis=0)
                pred  = np.concatenate((pred2, pred1), axis=0)
                clist.append(pred) 
            except:
                clist.append([])   
                
                
            try:
                pred1 = np.concatenate((x, y), axis=1)
                pred2 = np.flip(pred1, axis=1)
                pred  = np.concatenate((pred2, pred1), axis=0)
                clist.append(pred) 
            except:
                clist.append([])   # v384 ~ v895
                
    # ............................................................. 
    return clist

# ::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::: 
version.append(896)
solvers.append(f0002)


In [ ]:
inp, task, answer = get_arc(375,'eval')
clist = f0002(inp, task)
print('len clist:', len(clist))

for x in clist:
    if (np.array_equal(x, answer)):
        print('\nThis is correct:')
        plot_pic(x) 
        break

💡For **better visualization**, we randomly draw five more outputs from "clist".

In [ ]:
for i in range(5):
    n = random.randrange(len(clist))
    
    if (len(clist[n])!= 0):  # For when is not equal to []
        plot_pic(clist[n])    

<p style="border-bottom: 5px solid red"></p>

### ⭐Example 7

- This example is the Task No. 66 of the "training" file.

- In this example, the output is one third of the input image.

- But this simple example can be thought more deeply. For example, in the next solver, 52 zones from the input image are returned as output.

In [ ]:
hellow_arc(66,'train')

In [ ]:
# ::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::: 3  SelectZone
def f0003(inp, task):
    '''
    This function returns the selected zone.
    '''   
    clist = []
    ii = inp.shape[0]
    jj = inp.shape[1]
    
    uinp = np.unique(inp, return_counts= True)[0]
    n_uinp = np.unique(inp, return_counts= True)[1]
    
    uinp_min = uinp[np.argsort(n_uinp)[0]]
    uinp_max = uinp[np.argsort(n_uinp)[-1]]
    
    # ............................................................. 
    try:
        clist.append(np.array([[inp[0,0]]]))
    except:
        clist.append([])  # v0

    # ............................................................. 
    try:
        clist.append(np.array([[inp[-1,-1]]]))
    except:
        clist.append([])  # v1

    # ............................................................. 
    try:
        clist.append(np.array([[inp[0,-1]]]))
    except:
        clist.append([])  # v2

    # ............................................................. 
    try:
        clist.append(np.array([[inp[-1,0]]]))
    except:
        clist.append([])  # v3

    # ............................................................. 
    try:
        clist.append(np.array([inp[:,0]]))
    except:
        clist.append([])  # v4

    # ............................................................. 
    try:
        clist.append(np.array([inp[0,:]]))
    except:
        clist.append([])  # v5

    # ............................................................. 
    try:
        clist.append(np.array([inp[:,-1]]))
    except:
        clist.append([])  # v6

    # ............................................................. 
    try:
        clist.append(np.array([inp[-1,:]]))
    except:
        clist.append([])  # v7

    # ............................................................. 
    try:
        clist.append(np.array(inp[:2,:2]))
    except:
        clist.append([])  # v8

    # ............................................................. 
    try:
        clist.append(np.array(inp[:2,:]))
    except:
        clist.append([])  # v9

    # .............................................................  
    try:
        clist.append(np.array(inp[:,:2]))
    except:
        clist.append([])  # v10

    # ............................................................. 
    try:
        clist.append(np.array(inp[:ii//2 ,:]))
    except:
        clist.append([])  # v11

    # ............................................................. 
    try:
        clist.append(np.array(inp[ii//2: ,:]))
    except:
        clist.append([])  # v12

    # ............................................................. 
    try:
        clist.append(np.array(inp[: , :jj//2]))
    except:
        clist.append([])  # v13

    # ............................................................. 
    try:
        clist.append(np.array(inp[: , jj//2:]))
    except:
        clist.append([])  # v14

    # ............................................................. 
    try:
        clist.append(np.array(inp[:ii//2 , :jj//2]))
    except:
        clist.append([])  # v15

    # ............................................................. 
    try:
        clist.append(np.array(inp[ii//2: , :jj//2]))
    except:
        clist.append([])  # v16

    # .............................................................
    try:
        clist.append(np.array(inp[:ii//2 , jj//2:]))
    except:
        clist.append([])  # v17

    # ............................................................. 
    try:
        clist.append(np.array(inp[ii//2: , jj//2:]))
    except:
        clist.append([])  # v18

    # ............................................................. 
    try:
        clist.append(np.array(inp[-2:,-2:]))
    except:
        clist.append([])  # v19

    # ............................................................. 
    try:
        clist.append(np.array(inp[-2:,:]))
    except:
        clist.append([])  # v20

    # ............................................................. 
    try:
        clist.append(np.array(inp[:,-2:]))
    except:
        clist.append([])  # v21

    # ............................................................. 
    try:
        clist.append(np.array(inp[:3,:3]))
    except:
        clist.append([])  # v22

    # ............................................................. 
    try:
        clist.append(np.array(inp[:3,:]))
    except:
        clist.append([])  # v23

    # ............................................................. 
    try:
        clist.append(np.array(inp[:,:3]))
    except:
        clist.append([])  # v24

    # .............................................................         
    try:
        clist.append(np.array(inp[:ii//3 , :]))
    except:
        clist.append([])  # v25

    # .............................................................         
    try:
        clist.append(np.array(inp[: , :jj//3]))
    except:
        clist.append([])  # v26

    # ............................................................. 
    try:
        clist.append(np.array(inp[:ii//3 , :jj//3]))
    except:
        clist.append([])  # v27

    # ............................................................. 
    try:
        clist.append(np.array(inp[-3:,-3:]))
    except:
        clist.append([])  # v28

    # ............................................................. 
    try:
        clist.append(np.array(inp[-3:,:]))
    except:
        clist.append([])  # v29

    # ............................................................. 
    try:
        clist.append(np.array(inp[:,-3:]))
    except:
        clist.append([])  # v30

    # ............................................................. 
    try:
        leni = inp.shape[0]//2
        created = np.full((inp.shape), uinp_max)
        created[leni, :] = inp[leni, :] 
        
        clist.append(created)
    except:
        clist.append([])  # v31

    # ............................................................. 
    try:
        leni = inp.shape[0]//2
        created = np.full((inp.shape), 0)
        created[leni, :] = inp[leni, :] 
        
        clist.append(created)
    except:
        clist.append([])  # v32

    # ............................................................. 
    try:
        lenj = inp.shape[1]//2
        created = np.full((inp.shape), uinp_max)
        created[:, lenj] = inp[:, lenj] 
    
        clist.append(created)
    except:
        clist.append([])  # v33
        
    # ............................................................. 
    try:
        lenj = inp.shape[1]//2
        created = np.full((inp.shape), 0)
        created[:, lenj] = inp[:, lenj] 
    
        clist.append(created)
    except:
        clist.append([])  # v34       
        
    # .............................................................         
    try:
        leni = inp.shape[0]//2
        lenj = inp.shape[1]//2
        created = np.full((inp.shape), uinp_max)
        created[leni, :] = inp[leni, :] 
        created[:, lenj] = inp[:, lenj]
        
        clist.append(created)
    except:
        clist.append([])  # v35

    # .............................................................         
    try:
        leni = inp.shape[0]//2
        lenj = inp.shape[1]//2
        created = np.full((inp.shape), 0)
        created[leni, :] = inp[leni, :] 
        created[:, lenj] = inp[:, lenj]
        
        clist.append(created)
    except:
        clist.append([])  # v36
        
    # .............................................................         
    try:
        leni = inp.shape[0]//2
        lenj = inp.shape[1]//2
        created = np.full((inp.shape), uinp_max)
        created[leni, lenj] = inp[leni, lenj] 
        
        clist.append(created)
    except:
        clist.append([])  # v37

    # .............................................................         
    try:
        leni = inp.shape[0]//2
        lenj = inp.shape[1]//2
        created = np.full((inp.shape), 0)
        created[leni, lenj] = inp[leni, lenj] 
        
        clist.append(created)
    except:
        clist.append([])  # v38        
           
    # .............................................................         
    try:
        created = np.full((inp.shape), uinp_max)
        
        for x in range(inp.shape[0]):
            for y in range(inp.shape[1]):
                if (x == y):
                    created[x, y] = inp[x, y] 
                    
        clist.append(created)
    except:
        clist.append([])  # v39
        
    # .............................................................         
    try:
        created = np.full((inp.shape), 0)
        
        for x in range(inp.shape[0]):
            for y in range(inp.shape[1]):
                if (x == y):
                    created[x, y] = inp[x, y] 
                    
        clist.append(created)
    except:
        clist.append([])  # v40        
        
    # .............................................................         
    try:
        created = np.full((inp.shape), uinp_max)
        
        for x in range(inp.shape[0]):
            for y in range(inp.shape[1]):
                if (x == y):
                    created[x, -y-1] = inp[x, -y-1] 
                    
        clist.append(created)
    except:
        clist.append([])  # v41
        
    # .............................................................         
    try:
        created = np.full((inp.shape), 0)
        
        for x in range(inp.shape[0]):
            for y in range(inp.shape[1]):
                if (x == y):
                    created[x, -y-1] = inp[x, -y-1] 
                    
        clist.append(created)
    except:
        clist.append([])  # v42        
        
    # ............................................................. 
    try:
        created = np.full((inp.shape), uinp_max)
        
        for x in range(inp.shape[0]):
            for y in range(inp.shape[1]):
                if (x == y):
                    created[x, y] = inp[x, y]
                    created[x, -y-1] = inp[x, -y-1] 
                    
        clist.append(created)
    except:
        clist.append([])  # v43
        
    # .............................................................         
    try:
        created = np.full((inp.shape), 0)
        
        for x in range(inp.shape[0]):
            for y in range(inp.shape[1]):
                if (x == y):
                    created[x, y] = inp[x, y]
                    created[x, -y-1] = inp[x, -y-1] 
                    
        clist.append(created)
    except:
        clist.append([])  # v44      
        
    # .............................................................     
    try:
        leni = inp.shape[0]//2
        clist.append(np.array([inp[leni, :]]))
    except:
        clist.append([])  # v45

    # ............................................................. 
    try:
        lenj = inp.shape[1]//2
        clist.append(np.array([inp[:, lenj]]))
    except:
        clist.append([])  # v46

    # .............................................................         
    try:
        leni = inp.shape[0]//2
        lenj = inp.shape[1]//2
        clist.append(np.array([[inp[leni, lenj]]]))
    except:
        clist.append([])  # v47

    # .............................................................         
    try:
        created = np.full((inp.shape[0], 1), uinp_max)
        
        for x in range(inp.shape[0]):
            for y in range(inp.shape[1]):
                if (x == y):
                    created[x, 0] = inp[x, y] 
                    
        clist.append(created)
    except:
        clist.append([])  # v48
        
    # .............................................................         
    try:
        created = np.full((inp.shape[0], 1), 0)
        
        for x in range(inp.shape[0]):
            for y in range(inp.shape[1]):
                if (x == y):
                    created[x, 0] = inp[x, y] 
                    
        clist.append(created)
    except:
        clist.append([])  # v49       
        
    # .............................................................         
    try:
        created = np.full((inp.shape[0], 1), uinp_max)
        
        for x in range(inp.shape[0]):
            for y in range(inp.shape[1]):
                if (x == y):
                    created[x, 0] = inp[x, -y-1] 
                    
        clist.append(created)
    except:
        clist.append([])  # v50
        
    # .............................................................         
    try:
        created = np.full((inp.shape[0], 1), 0)
        
        for x in range(inp.shape[0]):
            for y in range(inp.shape[1]):
                if (x == y):
                    created[x, 0] = inp[x, -y-1] 
                    
        clist.append(created)
    except:
        clist.append([])  # v51
       
    # .............................................................    
    return clist

# ::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::: 
version.append(52)
solvers.append(f0003)        
               

In [ ]:
inp, task, answer = get_arc(66,'train')
clist = f0003(inp, task)
print('len clist:', len(clist))

for x in clist:
    if (np.array_equal(x, answer)):
        print('\nThis is correct:')
        plot_pic(x) 
        break

## <div style="color:yellow;display:inline-block;border-radius:10px;background-color:darkcyan;font-block:Nexa;overflow:hidden"><p style="padding:15px;color:white;overflow:hidden;font-size:95%;letter-spacing:0.5px;margin:0"><b> </b>🎯Practice</p></div>

- As **a practice**, you can create 8 new outputs for each of the 52 outputs using "Mirror&Rotate". Maybe another task can be solved in this way.

- For **better visualization**, we randomly draw five more outputs from "clist".

In [ ]:
for i in range(5):
    n = random.randrange(len(clist))
    
    if (len(clist[n])!= 0):  # For when is not equal to []
        plot_pic(clist[n])    

# <span style="color:darkcyan;">🔢Sudoku Puzzles</span>

<p style="border-bottom: 5px solid red"></p>
<p style="border-bottom: 15px solid darkcyan"></p>

- In **classic Sudoku**, the objective is to fill a 9 × 9 grid with digits so that each column, each row, and each of the nine 3 × 3 subgrids that compose the grid (also called "boxes", "blocks", or "regions") contains all of the digits from 1 to 9. The puzzle setter provides a partially completed grid, which for a well-posed puzzle has a single solution.

![](https://upload.wikimedia.org/wikipedia/commons/thumb/e/e0/Sudoku_Puzzle_by_L2G-20050714_standardized_layout.svg/800px-Sudoku_Puzzle_by_L2G-20050714_standardized_layout.svg.png)



<p style="border-bottom: 5px solid red"></p>

### ⭐Example 8 

- This example is the Task No. 66 of the  "evaluation"  file.

- This example is very similar to "Classic Sudoku".

- These types of examples are common in logical languages ​​and we think this example is suitable for this notebook as well. Of course, the solver we wrote for this example has only one answer, but we put it in the "clist" list to be consistent with the rest of the solvers.

In [ ]:
hellow_arc(115,'eval')

In [ ]:
# ::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::: 4  SudokuPuzzles
def f0004(inp, task):
    '''
    This function works like Sudoku puzzles.
    '''  
    clist = []
    
    # .............................................................
    inpz = np.array(task['train'][0]['input'])
    outz = np.array(task['train'][0]['output'])
            
    uinpz = np.unique(inpz, return_counts= True)[0]
    uoutz = np.unique(outz, return_counts= True)[0]
            
    not_inpz = [f for f in uoutz if f not in uinpz]
    not_outz = [f for f in uinpz if f not in uoutz]

    # ............................................................. 
    try:
        created = inp.copy()
    
        uinp = np.unique(inp, return_counts= True)[0]
        uinp = [f for f in uinp if (f!= not_outz[0])]
    
        for n in range(10):
            for x in range(inp.shape[0]):
                for y in range(inp.shape[1]):
            
                    if (created[x, y] == not_outz[0]):
                
                        setx = created[x, :]
                        usetx = np.unique(setx)
                        if (not_outz[0] in usetx):
                            if (len(usetx) == len(uinp)):
                                select = [f for f in uinp if (f not in usetx)]
                                created[x, :][created[x, :] ==  not_outz[0]] = select[0]
                        
                        sety = created[:, y]
                        usety = np.unique(sety)
                        if (not_outz[0] in usety):
                            if (len(usety) == len(uinp)):
                                select = [f for f in uinp if (f not in usety)]
                                created[:, y][created[:, y] ==  not_outz[0]] = select[0]                

        clist.append(created)   
    except:
        clist.append([])    # v17          
         
    # .............................................................  
    return clist

# ::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::: 
version.append(1)
solvers.append(f0004)


In [ ]:
inp, task, answer = get_arc(115,'eval')
clist = f0004(inp, task)

for x in clist:
    if (len(x)!= 0):  # For when is not equal to []
        print(np.array_equal(x, answer))
        plot_pic(x) 

# <span style="color:darkcyan;">🖼️Carpet Repair</span>

<p style="border-bottom: 5px solid red"></p>
<p style="border-bottom: 15px solid darkcyan"></p>

- **Carpet Repair**; It means tasks that have a variety of colors, but the color of some cells is unclear. Usually we are asked to specify the color of unknown cells.

- Different types of these tasks can be seen in "Corpus". We write suitable solvers for some of them, and you can solve the rest of the tasks with changes in these solvers.

<p style="border-bottom: 5px solid red"></p>

### ⭐Example 9

- This example is the Task No. 286 of the "training" file.

- It seems that in all pairs, the initial shape is symmetrical.

- In this task, after completing the initial shape, the whole shape should be returned.

In [ ]:
hellow_arc(286,'train')

<p style="border-bottom: 5px solid red"></p>

### ⭐Example 10

- This example is the Task No. 16 of the "training" file.

- The original shape is not symmetrical.

- In this task, after completing the initial shape, the whole shape should be returned.

In [ ]:
hellow_arc(16,'train')

<p style="border-bottom: 5px solid red"></p>

### ⭐Example 11

- This example is the Task No. 304 of the "training" file.

- The original shape is not symmetrical.

- In this task, after completing the initial shape, the whole shape should be returned.

In [ ]:
hellow_arc(304,'train')

<p style="border-bottom: 5px solid red"></p>

### ⭐Example 12

- This example is the Task No. 383 of the "evaluation"" file.

- It seems that in all pairs, the initial shape is symmetrical.

- In this task, after completing the initial form, only the completed part should be returned.

In [ ]:
hellow_arc(383,'eval')

- To solve the above examples, we first write two solvers using Mirror&Rotate.

- But unfortunately, we will see that this method is not suitable for all examples. For this reason, we modify and complete the solvers.

In [ ]:
# :::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::
def ft3(inp, task):
    '''
    This function performs "Carpet Repair" by flipping and rotating.
    ''' 
    inpz = np.array(task['train'][0]['input'])
    outz = np.array(task['train'][0]['output'])
            
    uinpz = np.unique(inpz, return_counts= True)[0]
    uoutz = np.unique(outz, return_counts= True)[0]
            
    not_inpz = [f for f in uoutz if f not in uinpz]
    not_outz = [f for f in uinpz if f not in uoutz]

    # ............................................................. 
    try: 
        created = inp.copy()
        c_change = not_outz[0]
        seet = np.flip(np.rot90(inp, 1), axis=0)
    
        for x in range(inp.shape[0]):
            for y in range(inp.shape[1]):
            
                if (created[x, y]== c_change) and (seet[x, y]!= c_change):
                    created[x, y] = seet[x, y]
    except:
        created = []
        
    return created

# :::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::
def ft4(inp, task):
    '''
    This function performs "Carpet Repair" by flipping and rotating.
    ''' 
    inpz = np.array(task['train'][0]['input'])
    outz = np.array(task['train'][0]['output'])
            
    uinpz = np.unique(inpz, return_counts= True)[0]
    uoutz = np.unique(outz, return_counts= True)[0]
            
    not_inpz = [f for f in uoutz if f not in uinpz]
    not_outz = [f for f in uinpz if f not in uoutz]

    # ............................................................. 
    try: 
        created = inp.copy()
        c_change = not_outz[0]
        seet = np.flip(np.rot90(inp, 3), axis=0)
    
        for x in range(inp.shape[0]):
            for y in range(inp.shape[1]):
            
                if (created[x, y]== c_change) and (seet[x, y]!= c_change):
                    created[x, y] = seet[x, y]           
    except:
        created = []
        
    return created

# :::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::

In [ ]:
# Example 9
inp, task, answer = get_arc(286,'train')
created = ft3(inp, task)

print(np.array_equal(created, answer))
plot_pic(created) 

In [ ]:
# Example 9
inp, task, answer = get_arc(286,'train')
created = ft4(inp, task)

print(np.array_equal(created, answer))
plot_pic(created) 

- That is, if we rotate the symmetric image in the right direction, it will be solved with these few lines of code.

In [ ]:
# Example 10
inp, task, answer = get_arc(16,'train')
created = ft3(inp, task)

print(np.array_equal(created, answer))
plot_pic(created) 

In [ ]:
# Example 10
inp, task, answer = get_arc(16,'train')
created = ft4(inp, task)

print(np.array_equal(created, answer))
plot_pic(created) 

In [ ]:
# Example 11
inp, task, answer = get_arc(304,'train')
created = ft3(inp, task)

print(np.array_equal(created, answer))
plot_pic(created) 

In [ ]:
# Example 11
inp, task, answer = get_arc(304,'train')
created = ft4(inp, task)

print(np.array_equal(created, answer))
plot_pic(created) 

- As you can see, many images cannot be solved by simple rotations.

- The solution we present below is a combination of proper rotations as well as comparing all rows and columns. Of course, we do this at least twice. This solver has similarities with the Sudoku solver that you have seen before.

In [ ]:
# ::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::: 5  CarpetRepair_1
def f0005(inp, task):
    '''
    This function performs a "Carpet Repair" and after completing the initial shape, returns both the whole shape and the completed part.
    '''  
    clist = []
    
    # .............................................................
    inpz = np.array(task['train'][0]['input'])
    outz = np.array(task['train'][0]['output'])
            
    uinpz = np.unique(inpz, return_counts= True)[0]
    uoutz = np.unique(outz, return_counts= True)[0]
            
    not_inpz = [f for f in uoutz if f not in uinpz]
    not_outz = [f for f in uinpz if f not in uoutz]

    # ............................................................. 
    try: 
        created = inp.copy()
        c_change = not_outz[0]
        
        uinp = np.unique(inp)
        wcolor = np.argwhere(inp == c_change)
      
        wrow, wcol = [], []
        for w in wcolor:
            wrow.append(w[0])
            wcol.append(w[1])
        
        minr = min(wrow)
        maxr = max(wrow)
        minc = min(wcol)
        maxc = max(wcol) 
    
        # .........................................................
        seet = np.flip(np.rot90(inp, 1), axis=0)
    
        for x in range(inp.shape[0]):
            for y in range(inp.shape[1]):
            
                if (created[x, y]== c_change) and (seet[x, y]!= c_change):
                    created[x, y] = seet[x, y]     
                        
        # .........................................................
        for i in range(created.shape[0]):
            if c_change in created[i, :]:  
            
                rowi = created[i, :]
                arg_change   = np.argwhere(rowi == c_change).ravel()
                arg_unchange = np.argwhere(rowi != c_change).ravel()
              
                changei   = rowi[arg_change]
                unchangei = rowi[arg_unchange]
            
                for ii in range(created.shape[0]):
                
                    rowii = created[ii, :]
                    changeii   = rowii[arg_change]
                    unchangeii = rowii[arg_unchange]
                
                    if (np.array_equal(unchangei, unchangeii)):
                        created[i, :] = rowii
                
        for j in range(created.shape[1]):
            if c_change in created[:, j]:  
            
                colj = created[:, j]
                arg_change   = np.argwhere(colj == c_change).ravel()
                arg_unchange = np.argwhere(colj != c_change).ravel()
              
                changej   = colj[arg_change]
                unchangej = colj[arg_unchange]
            
                for jj in range(created.shape[1]):
                
                    coljj = created[:, jj]
                    changejj   = coljj[arg_change]
                    unchangejj = coljj[arg_unchange]
                
                    if (np.array_equal(unchangej, unchangejj)):
                        created[:, j] = coljj
                    
        # .........................................................
        for i in range(created.shape[0]):
            if c_change in created[i, :]:  
            
                rowi = created[i, :]
                arg_change   = np.argwhere(rowi == c_change).ravel()
                arg_unchange = np.argwhere(rowi != c_change).ravel()
              
                changei   = rowi[arg_change]
                unchangei = rowi[arg_unchange]
            
                for ii in range(created.shape[0]):
                
                    rowii = created[ii, :]
                    changeii   = rowii[arg_change]
                    unchangeii = rowii[arg_unchange]
                
                    if (np.array_equal(unchangei, unchangeii)):
                        created[i, :] = rowii
                
        for j in range(created.shape[1]):
            if c_change in created[:, j]:  
            
                colj = created[:, j]
                arg_change   = np.argwhere(colj == c_change).ravel()
                arg_unchange = np.argwhere(colj != c_change).ravel()
              
                changej   = colj[arg_change]
                unchangej = colj[arg_unchange]
            
                for jj in range(created.shape[1]):
                
                    coljj = created[:, jj]
                    changejj   = coljj[arg_change]
                    unchangejj = coljj[arg_unchange]
                
                    if (np.array_equal(unchangej, unchangejj)):
                        created[:, j] = coljj
                    
        # .............................................................          
        clist.append(created)   
        created = created[minr:maxr+1 , minc:maxc+1]  
        clist.append(created) 
    except:
        clist.append([])
        clist.append([])

    # .................................................................  
    return clist

# ::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::: 
version.append(2)
solvers.append(f0005)               


In [ ]:
# Example 9
inp, task, answer = get_arc(286,'train')
clist = f0005(inp, task)

for x in clist:
    if (np.array_equal(x, answer)):
        print('\nThis is correct:')
        plot_pic(x) 

In [ ]:
# Example 10
inp, task, answer = get_arc(16,'train')
clist = f0005(inp, task)

for x in clist:
    if (np.array_equal(x, answer)):
        print('\nThis is correct:')
        plot_pic(x) 

In [ ]:
# Example 11
inp, task, answer = get_arc(304,'train')
clist = f0005(inp, task)

for x in clist:
    if (np.array_equal(x, answer)):
        print('\nThis is correct:')
        plot_pic(x) 

In [ ]:
# Example 12
inp, task, answer = get_arc(383,'eval')
clist = f0005(inp, task)

for x in clist:
    if (np.array_equal(x, answer)):
        print('\nThis is correct:')
        plot_pic(x) 

<p style="border-bottom: 5px solid red"></p>

### ⭐Example 13

- This example is the Task No. 391 of the "evaluation" file.

- The original shape is not symmetrical.

- In this task, after completing the initial shape, the whole shape should be returned.

In [ ]:
hellow_arc(391,'eval')

In [ ]:
# Example 13
inp, task, answer = get_arc(391,'eval')
clist = f0005(inp, task)

for x in clist:
    if (np.array_equal(x, answer)):
        print('\nThis is correct:')
        plot_pic(x)  
    else:
        print('\nThis is not correct:')
        plot_pic(x)   

- Only one cell is wrong, but it will not be scored.

### 💡Inference

- Please note that some tasks are solved only if the rotation method is performed at the end. We wrote the following solver exactly based on this topic.

- Of course, adding this solver creates repeated correct results for some tasks, which is definitely not a problem.

In [ ]:
# ::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::: 6  CarpetRepair_2
def f0006(inp, task):
    '''
    This function performs a "Carpet Repair" and after completing the initial shape, returns both the whole shape and the completed part.
    '''  
    clist = []
    
    # .............................................................
    inpz = np.array(task['train'][0]['input'])
    outz = np.array(task['train'][0]['output'])
            
    uinpz = np.unique(inpz, return_counts= True)[0]
    uoutz = np.unique(outz, return_counts= True)[0]
            
    not_inpz = [f for f in uoutz if f not in uinpz]
    not_outz = [f for f in uinpz if f not in uoutz]

    # ............................................................. 
    try: 
        created = inp.copy()
        c_change = not_outz[0]
        
        uinp = np.unique(inp)
        wcolor = np.argwhere(inp == c_change)
      
        wrow, wcol = [], []
        for w in wcolor:
            wrow.append(w[0])
            wcol.append(w[1])
        
        minr = min(wrow)
        maxr = max(wrow)
        minc = min(wcol)
        maxc = max(wcol) 
    
        # .........................................................
        for i in range(created.shape[0]):
            if c_change in created[i, :]:  
            
                rowi = created[i, :]
                arg_change   = np.argwhere(rowi == c_change).ravel()
                arg_unchange = np.argwhere(rowi != c_change).ravel()
              
                changei   = rowi[arg_change]
                unchangei = rowi[arg_unchange]
            
                for ii in range(created.shape[0]):
                
                    rowii = created[ii, :]
                    changeii   = rowii[arg_change]
                    unchangeii = rowii[arg_unchange]
                
                    if (np.array_equal(unchangei, unchangeii)):
                        created[i, :] = rowii
                
        for j in range(created.shape[1]):
            if c_change in created[:, j]:  
            
                colj = created[:, j]
                arg_change   = np.argwhere(colj == c_change).ravel()
                arg_unchange = np.argwhere(colj != c_change).ravel()
              
                changej   = colj[arg_change]
                unchangej = colj[arg_unchange]
            
                for jj in range(created.shape[1]):
                
                    coljj = created[:, jj]
                    changejj   = coljj[arg_change]
                    unchangejj = coljj[arg_unchange]
                
                    if (np.array_equal(unchangej, unchangejj)):
                        created[:, j] = coljj
                    
        # .........................................................
        for i in range(created.shape[0]):
            if c_change in created[i, :]:  
            
                rowi = created[i, :]
                arg_change   = np.argwhere(rowi == c_change).ravel()
                arg_unchange = np.argwhere(rowi != c_change).ravel()
              
                changei   = rowi[arg_change]
                unchangei = rowi[arg_unchange]
            
                for ii in range(created.shape[0]):
                
                    rowii = created[ii, :]
                    changeii   = rowii[arg_change]
                    unchangeii = rowii[arg_unchange]
                
                    if (np.array_equal(unchangei, unchangeii)):
                        created[i, :] = rowii
                
        for j in range(created.shape[1]):
            if c_change in created[:, j]:  
            
                colj = created[:, j]
                arg_change   = np.argwhere(colj == c_change).ravel()
                arg_unchange = np.argwhere(colj != c_change).ravel()
              
                changej   = colj[arg_change]
                unchangej = colj[arg_unchange]
            
                for jj in range(created.shape[1]):
                
                    coljj = created[:, jj]
                    changejj   = coljj[arg_change]
                    unchangejj = coljj[arg_unchange]
                
                    if (np.array_equal(unchangej, unchangejj)):
                        created[:, j] = coljj
         
        # .........................................................
        seet = np.flip(np.rot90(inp, 1), axis=0)
    
        for x in range(inp.shape[0]):
            for y in range(inp.shape[1]):
            
                if (created[x, y]== c_change) and (seet[x, y]!= c_change):
                    created[x, y] = seet[x, y]     
                          
    # .................................................................           
        clist.append(created)   
        created = created[minr:maxr+1 , minc:maxc+1]  
        clist.append(created) 
    except:
        clist.append([])
        clist.append([])

    # .................................................................  
    return clist

# ::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::: 
version.append(2)
solvers.append(f0006)               


In [ ]:
# Example 13
inp, task, answer = get_arc(391,'eval')
clist = f0006(inp, task)

for x in clist:
    if (np.array_equal(x, answer)):
        print('\nThis is correct:')
        plot_pic(x)  

# <span style="color:darkcyan;">⚖️Identical Columns & Rows</span>

<p style="border-bottom: 5px solid red"></p>
<p style="border-bottom: 15px solid darkcyan"></p>

- Removing identical columns and rows in an image can be the goal of a task.

- But sometimes deleting identical columns and rows can be very interesting. That is, it creates unpredictable changes in the original image. Pay attention to the example below.

<p style="border-bottom: 5px solid red"></p>

### ⭐Example 14

- This example is the Task No. 376 of the "training" file.

- To solve this task, it is enough to remove identical columns and identical rows in each input image.

- First, we wrote a primitive solver for this example, and then we wrote a solution that covers other cases as well.

In [ ]:
hellow_arc(376,'train')

In [ ]:
# :::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::
def ft5(inp, task):
    '''
    This function removes identical columns and identical rows.
    ''' 
    try: 
        ilist = []
        if (inp.shape[0] > 1):
            for i in range(1, inp.shape[0]):
                if all(inp[i-1, :] == inp[i, :]):
                    ilist.append(i-1)
                    
        jlist = []
        if (inp.shape[1] > 1):
            for j in range(1, inp.shape[1]):
                if all(inp[:, j-1] == inp[:, j]):
                    jlist.append(j-1)
                
        created = inp.copy()
        created = np.delete(created, ilist, 0)
        created = np.delete(created, jlist, 1)           
        
    except:
        created = []
        
    return created

# :::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::

In [ ]:
# Example 14
inp, task, answer = get_arc(376,'train')
created = ft5(inp, task)

print(np.array_equal(created, answer))
plot_pic(created) 

- The following solver covers other modes as well.

In [ ]:
# ::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::: 7 Removes Identical Cols&Rows
def f0007(inp, task):
    '''
    This function removes identical columns and identical rows.
    '''   
    alist = []
    clist = []
    
    uinp = np.unique(inp, return_counts= True)[0]
    n_uinp = np.unique(inp, return_counts= True)[1]
    
    uinp_min = uinp[np.argsort(n_uinp)[0]] 
    uinp_max = uinp[np.argsort(n_uinp)[-1]]
     
    # .............................................................   
    try:
        ilist = []
        if (inp.shape[0] > 1):
            for i in range(1, inp.shape[0]):
                if all(inp[i-1, :] == inp[i, :]):
                    ilist.append(i-1)
                    
        jlist = []
        if (inp.shape[1] > 1):
            for j in range(1, inp.shape[1]):
                if all(inp[:, j-1] == inp[:, j]):
                    jlist.append(j-1)
                
        created = inp.copy()
        created = np.delete(created, ilist, 0)
        created = np.delete(created, jlist, 1)              

        alist.append(created) 
    except:
        alist.append([])    # delete identical rows&cols
           
    # .............................................................   
    try:
        ilist = []
        if (inp.shape[0] > 1):
            for i in range(1, inp.shape[0]):
                if all(inp[i-1, :] == inp[i, :]):
                    ilist.append(i-1)
                
        created = inp.copy()
        created = np.delete(created, ilist, 0)
        
        alist.append(created)
        
    except:
        alist.append([])    # delete identical rows
           
    # .............................................................       
    try:                
        jlist = []
        if (inp.shape[1] > 1):
            for j in range(1, inp.shape[1]):
                if all(inp[:, j-1] == inp[:, j]):
                    jlist.append(j-1)
                
        created = inp.copy()
        created = np.delete(created, jlist, 1)    

        alist.append(created)
        
    except:
        alist.append([])    # delete identical cols
           
    # .............................................................   
    try:
        ilist = []
        if (inp.shape[0] > 1):
            for i in range(1, inp.shape[0]):
                if all(inp[i-1, :] == uinp[0]):
                    ilist.append(i-1)
                    
        jlist = []
        if (inp.shape[1] > 1):
            for j in range(1, inp.shape[1]):
                if all(inp[:, j-1] == uinp[0]):
                    jlist.append(j-1)
                
        created = inp.copy()
        created = np.delete(created, ilist, 0)
        created = np.delete(created, jlist, 1)     

        alist.append(created)
        
    except:
        alist.append([])    # delete all-Zero rows&cols
               
    # .............................................................   
    try:
        ilist = []
        if (inp.shape[0] > 1):
            for i in range(1, inp.shape[0]):
                if all(inp[i-1, :] == uinp_max):
                    ilist.append(i-1)
                    
        jlist = []
        if (inp.shape[1] > 1):
            for j in range(1, inp.shape[1]):
                if all(inp[:, j-1] == uinp_max):
                    jlist.append(j-1)
                
        created = inp.copy()
        created = np.delete(created, ilist, 0)
        created = np.delete(created, jlist, 1)  

        alist.append(created)
        
    except:
        alist.append([])    # delete uinp_max rows&cols
           
    # .............................................................    
    for x in alist:
        
        if (len(x)!= 0):
            blist = f0000(x, [])
            for y in blist:
                clist.append(y) 
        else:
            for n in range(8):
                clist.append([])
     
    # .............................................................         
    return clist

# :::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::
version.append(40)
solvers.append(f0007)
    

In [ ]:
# Example 14
inp, task, answer = get_arc(376,'train')
clist = f0007(inp, task)

for x in clist:
    if (np.array_equal(x, answer)):
        print('\nThis is correct:')
        plot_pic(x)  
        break

# <span style="color:darkcyan;">🔀Setting tasks and solvers</span>

<p style="border-bottom: 5px solid red"></p>
<p style="border-bottom: 15px solid darkcyan"></p>

- With this function, we solve all corpus tasks (training file and evaluation file) with all available solvers (that is, all versions of all solvers) and return only the correct answers.

- All the correct answers are drawn, even if they are duplicates (for example, solved by several different versions), but only the correct answers are counted as unique.

- The number of solvers is still very small, but about five percent of the tasks in the corpus can be solved with these solvers.

- This is the beginning and for now we are using the "Brute Force" method. But this method should give way to a smart method. It is only then that we have taken a serious step towards learning "Abstraction and Reasoning" in machines.

- In other words, we first create a number of keys (solvers) and then to open (solve) each task, the machine must be able to choose the right key (or a number of keys that are more suitable) among all the keys. But in the "Brute Force" method, all keys are tested to find the right key, if any.


In [ ]:
# :::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::
def set_tasks_solvers(data1_path, data2_path, data_name): 
    
    with open(data1_path,'r') as f:
        tasks_name = list(json.load(f).keys())
    
    with open(data1_path,'r') as f:
        tasks_file = list(json.load(f).values())    
    
    with open(data2_path,'r') as f:
        tasks2_file = list(json.load(f).values())    
            
    # .................................................................................  
    print(f'{Style.BRIGHT}{Fore.CYAN}_' * 60)
    print('\n\t | Answers of solved tasks |', data_name, '|')
    print('_' * 60, '\n')   
    
    # .................................................................................  
    answer = []
    for n in range(len(tasks_file)):  
        task = tasks_file[n] 
   
        for sol in range(len(solvers)):
            for i in range(len(task['test'])):
                
                inp_test = np.array(task['test'][i]['input'])
                out_test = tasks2_file[n][i] 
                
                clist = solvers[sol](inp_test, task)
                for v in range(version[sol]):
                    
                    if (np.array_equal(clist[v], out_test)):
                        answer_ = clist[v].tolist()
                        
                        if (answer_ not in answer):
                            answer.append(answer_) 
                    
                        display(pd.DataFrame(data={'Answers for task': tasks_name[n],'Items': i,'Files': data_name},index=[n]))
                        display(pd.DataFrame(data={'The solver of this task': f'Function f000{sol}','version': v},index=[n]))
                    
                        plot_pic(answer_) 
                        print('\n', f'{Style.BRIGHT}{Fore.CYAN}|§|'*10, '✅✅✅', '|§|'*10, '\n')  
                        
                        break
                    
    # ............................................................................................................. 
    print(f'{Style.BRIGHT}{Fore.YELLOW}_' * 30)
    print('\nNumber of tasks solved :', len(answer))
    print(f'{Style.BRIGHT}{Fore.YELLOW}_' * 30)
    print(f'{Style.BRIGHT}{Fore.YELLOW}_' * 20)
    print(f'{Style.BRIGHT}{Fore.YELLOW}_' * 10)
    print()
    
# :::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::

set_tasks_solvers(train1_path, train2_path, 'Training File')

set_tasks_solvers(eval1_path, eval2_path, 'Evaluation File')  

# :::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::

# <span style="color:darkcyan;">🔀Setting & Submission</span>

<p style="border-bottom: 5px solid red"></p>
<p style="border-bottom: 15px solid darkcyan"></p>

- By solvers and this function, we solve all the tasks in the "testing" file and return the correct answers.

- To choose the correct answers in the "testing" file, great care must be taken. Because we don't have access to the answer like the "training" file or the "evaluation" file. The only way is to use train (input/output) pairs of that task and find the correct answer.

- The logical argument is that if a solver (key) is suitable for solving (opening) all train (input/output) pairs and always returns the correct answer, then this solver is the one we are looking for. So if we give "input_test" to this solver, it will definitely return the correct answer.

- The above argument is only valid if all tasks are "standard". It means that the number of pairs of trains for each task should be sufficient and no ifs and buts about it. Even all questions of human intelligence must be "standard", otherwise even humans cannot find the correct answer.

- In this function, we first assumed that all tasks in the "testing" file are standard and entered the correct answers in ['attempt_1']. But it seems that there are still non-standard questions. So, we filled ['attempt_2'] with answers that only one of the train pairs did not have the correct answer.



In [ ]:
# :::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::
def set_submission(): 
    
    with open(sample_path,'r') as f:
        sample_sub = json.load(f) 
    
    with open(sample_path,'r') as f:
        sample_name = list(json.load(f).keys())
    
    # ................................................................................. 
    with open(test_path,'r') as f:
        tasks_name = list(json.load(f).keys())
    
    with open(test_path,'r') as f:
        tasks_file = list(json.load(f).values())    

    # .................................................................................  
    print(f'{Style.BRIGHT}{Fore.CYAN}_' * 60)
    print('\n\t | Answers of solved tasks |', 'Testing', '|')
    print('_' * 60, '\n')   
    
    # .................................................................................  
    answer = []
    for n in range(len(tasks_file)): 
        
        task = tasks_file[n] 
        len_train = len(task['train'])
        
        for sol in range(len(solvers)):
            counter = np.zeros(version[sol])
        
            for e in range(len_train):
                inp_e = np.array(task['train'][e]['input'])
                out_e = np.array(task['train'][e]['output'])
            
                try:
                    clist = solvers[sol](inp_e, task)
                    
                    for v in range(version[sol]): 
                        if (np.array_equal(clist[v], out_e)):
                            counter[v] += 1          
                except:
                    counter = counter       
            
            # .............................................................................................................
            for v in range(len(counter)):
                
                if (counter[v] == len_train):
            
                    for i in range(len(task['test'])):
                        inp_test = np.array(task['test'][i]['input'])
                        
                        clist = solvers[sol](inp_test, task)
                        answer_ = clist[v].tolist()
                        
                        sample_sub[tasks_name[n]][i]['attempt_1'] = answer_
                        if (answer_ not in answer):
                            answer.append(answer_) 
                        
                        display(pd.DataFrame(data={'Answers for task': tasks_name[n],'Items': i,'Attempt':'1','Files': 'Testing'},index=[n]))
                        print('\nNumber of correct answers for train_inputs of this task :', counter[v], '(All)')
                        display(pd.DataFrame(data={'The solver of this task': f'Function f000{sol}','version': v},index=[n]))
                        
                        plot_pic(answer_) 
                        print('\n', f'{Style.BRIGHT}{Fore.CYAN}|§|'*10, '✅✅✅', '|§|'*10, '\n')  
                        
                    break

                # .............................................................................................................  
                if (counter[v] == len_train-1):
                    
                    for i in range(len(task['test'])):
                        inp_test = np.array(task['test'][i]['input'])
                        
                        try:
                            clist = solvers[sol](inp_test, task)
                            answer_ = clist[v].tolist()
                            
                            sample_sub[tasks_name[n]][i]['attempt_2'] = answer_
                            if (answer_ not in answer):
                                answer.append(answer_) 
                            
                            display(pd.DataFrame(data={'Answers for task': tasks_name[n],'Items': i,'Attempt':'2','Files': 'Testing'},index=[n]))
                            print('\nNumber of correct answers for train_inputs of this task :', counter[v]) 
                            display(pd.DataFrame(data={'The solver of this task': f'Function f000{sol}','version': v},index=[n]))
                            
                            plot_pic(answer_) 
                            print('\n', f'{Style.BRIGHT}{Fore.YELLOW}|§|'*10, '☑️☑️☑️', '|§|'*10, '\n') 
                            
                        except:
                            break  
                    break 
                          
    # ............................................................................................................. 
    print(f'{Style.BRIGHT}{Fore.YELLOW}_' * 30)
    print('\nNumber of tasks solved :', len(answer))
    print(f'{Style.BRIGHT}{Fore.YELLOW}_' * 30)
    print(f'{Style.BRIGHT}{Fore.YELLOW}_' * 20)
    print(f'{Style.BRIGHT}{Fore.YELLOW}_' * 10)
    print()
    
    return sample_sub
    
# :::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::

sample_sub = set_submission()

with open('submission.json', 'w') as file:
    json.dump(sample_sub, file, indent=4)

# :::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::

## <div style="color:yellow;display:inline-block;border-radius:10px;background-color:darkcyan;font-block:Nexa;overflow:hidden"><p style="padding:15px;color:white;overflow:hidden;font-size:95%;letter-spacing:0.5px;margin:0"><b> </b>🎯Practice - Solution</p></div>

- We hope you have already solved the practice of this notebook.

- However, we also provide a solution below to show that you can easily add this solver, as well as any other solvers you write yourself, to the list of solvers in this notebook.

In [ ]:
# ::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::: 8  SelectZone & RotationFlip
def f0008(inp, task):
    '''
    This function returns the selected zone & RotationFlip.
    '''   
    clist = []
    alist = f0003(inp, task)
    
    for x in alist:
        try: 
            blist = f0000(x, [])
            for y in blist:
                clist.append(y)      
        except: 
            for i in range(8):
                clist.append([])
                
    # .............................................................    
    return clist

# ::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::: 
version.append(416)
solvers.append(f0008)           
    

In [ ]:
# :::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::

# sample_sub = set_submission()

# with open('submission.json', 'w') as file:
    # json.dump(sample_sub, file, indent=4)

# :::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::

![](https://cdn-images-1.medium.com/max/1000/1*fcnycT92eQYAyH8jeaJzUw.png)

<p style="border-bottom: 5px solid red"></p>
<p style="border-bottom: 15px solid darkcyan"></p>